# Capstone Project - The Battle of the Neighborhoods 
### Applied Data Science Capstone by IBM/Coursera

### Selecting the Best Neighborhood in Los Angeles to open a new Ethiopian Restaurant

## Introduction
Running a successful restaurant business in any city is hard. Most restaurants close within a short period of time after opening, because they are not profitable. Therefore, considering all the factors that affect the success of the restaurant is important. One of the important factors that affect the restaurant business is its location. In this project, we will try to find the best location to open a new Ethiopian restaurant in Los Angeles, CA. We consider the density of other restaurants in the neighborhoods, and next the density of Ethiopian restaurants in the neighborhoods. So we would not have many competitors surrounding our restaurant. This problem is important to solve for stakeholders, such as any person who is interested in opening a new Ethiopian restaurant in Los Angeles.

## Data
We will use three data sources to solve the problem.

1) We will get the list of neighborhoods of Los Angeles, CA from an excel file that we got by scraping a Wikipedia page.        
2) We will get Latitude and Longitude coordinates of the neighborhoods, in order to plot maps and use the coordinates to get venue data. We will use Python Geocoder package to get the coordinates.                                                        
3) We will get venue data, especially restaurants and particularly Ethiopian restaurants, in these neighborhoods using Foursquare API. We will use this data to perform clustering on the neighborhoods.

 ### Import the necessary packages

In [45]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import numpy as np

In [46]:
import geocoder
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm 
import matplotlib.colors as colors


#import k-means from clustering stage
from sklearn.cluster import KMeans

import folium #map rendering library

In [47]:
# open and read an excel file containing Los Angeles Neighborhoods (extracted from wikipedia page) and name the dataframe 'la_data' and see sample of the contents using head() function
la_data = pd.read_excel(r'C:\Users\Mike\Desktop\Los Angeles Neighborhoods .xlsx')
print(la_data.shape)
la_data.head()

(201, 1)


,Neighborhood
0,Angelino Heights
1,Angeles Mesa
2,Angelus Vesta
3,Arleta
4,Arlington Heights


In [48]:
# Defining a function to get coordinates of the neighborhoods
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Los Angeles, US'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in la_data["Neighborhood"].tolist()]

In [49]:
coords[0:10]

[[34.07029000000006, -118.25479999999999],
 [18.285830000000033, -66.80088999999998],
 [33.994983237346126, -118.34111670625835],
 [34.24905000000007, -118.43348999999995],
 [34.03989000000007, -118.32515999999998],
 [34.041964000000064, -118.23638699999998],
 [34.11970000000008, -118.25886999999994],
 [34.021570000000054, -118.36764999999997],
 [34.01009400000004, -118.337336],
 [34.07044548680411, -118.20070975126488]]

In [50]:
# Create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
la_data['Latitude'] = df_coords['Latitude']
la_data['Longitude'] = df_coords['Longitude']
print(la_data.shape)
la_data.head()

(201, 3)


,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070290,-118.254800
1,Angeles Mesa,18.285830,-66.800890
2,Angelus Vesta,33.994983,-118.341117
3,Arleta,34.249050,-118.433490
4,Arlington Heights,34.039890,-118.325160


In [51]:
# use geopy to get the latitudes and longitudes of Los Angeles
address = 'Los Angeles, US'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles, US are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles, US are 34.0536909, -118.242766.


In [52]:
# create map of Los Angeles using the latitude and longitude values
map_LosAngeles = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers to map
for lat, lng, neighborhood in zip(la_data['Latitude'], la_data['Longitude'], la_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_LosAngeles)  
    
map_LosAngeles

In [53]:
# use foursquare API to explore neighborhoods
CLIENT_ID = 'TB1CA3QB3CCLCMRUHUERFQTGH4LCUVZPDL2MJGHIJKFWUPPZ' # your Foursquare ID
CLIENT_SECRET = 'N2HE3QO15CDIVZ4RK30WFC2R4BUAUWEI2O2RDAZTSV1Y1UE5' # your Foursquare Secret
VERSION = '20210131' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TB1CA3QB3CCLCMRUHUERFQTGH4LCUVZPDL2MJGHIJKFWUPPZ
CLIENT_SECRET:N2HE3QO15CDIVZ4RK30WFC2R4BUAUWEI2O2RDAZTSV1Y1UE5


In [54]:
# let's get one neighborhood to explore using foursquare API to find the top 100 venues within it
# let's pick the first neighborhood in the dataframe, 'Angelino Heights' and name its latitude and longitude as neigh_lat and neigh_long respectively
neigh_lat = la_data.loc[0, 'Latitude']
neigh_long = la_data.loc[0,'Longitude']
neigh_name = la_data.loc[0, 'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_long))



Latitude and longitude values of Angelino Heights are 34.07029000000006, -118.25479999999999.


In [55]:
# now let's get the top 100 venues that are in 'Angelino Heights' within a radius of 500 meters
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neigh_lat, neigh_long, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=TB1CA3QB3CCLCMRUHUERFQTGH4LCUVZPDL2MJGHIJKFWUPPZ&client_secret=N2HE3QO15CDIVZ4RK30WFC2R4BUAUWEI2O2RDAZTSV1Y1UE5&ll=34.07029000000006,-118.25479999999999&v=20210131&radius=500&limit=100'

In [56]:
# send the GET request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '603adbd38fda8b55522776ee'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'East LA',
  'headerFullLocation': 'East LA, Los Angeles',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 26,
  'suggestedBounds': {'ne': {'lat': 34.07479000450006,
    'lng': -118.24937765750715},
   'sw': {'lat': 34.065789995500054, 'lng': -118.26022234249282}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f75a626e5e8f16c87566797',
       'name': 'Halliwell Manor',
       'location': {'address': '1329 Carroll Ave',
        'lat': 34.069328534140894,
        'lng': -118.25416524263122,
        'labeledLatLngs'

We know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [57]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [58]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

C:\Users\Mike\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Halliwell Manor,Performing Arts Venue,34.069329,-118.254165
1,Guisados,Taco Place,34.070262,-118.250437
2,Eightfold Coffee,Coffee Shop,34.071245,-118.250698
3,"Michael Jackson's ""Thriller"" House (and Tree)",Historic Site,34.069557,-118.254599
4,Ototo,Sake Bar,34.072659,-118.251740
5,Subliminal Projects,Art Gallery,34.072290,-118.250737
6,Tsubaki,Japanese Restaurant,34.072938,-118.251298
7,The Park's Finest BBQ,BBQ Joint,34.066519,-118.254291
8,K Bakery,Bakery,34.066124,-118.254059
9,The Park Restaurant,Breakfast Spot,34.074243,-118.252242


In [59]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

26 venues were returned by Foursquare.


## Explore Neighborhoods in Los Angeles


#### Let's create a function to repeat the same process to all the neighborhoods in Los Angeles


In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _losangeles_venues_.


In [61]:
losangeles_venues= getNearbyVenues(names= la_data['Neighborhood'],
                                 latitudes= la_data['Latitude'],
                                 longitudes= la_data['Longitude']
                                 )

Angelino Heights
Angeles Mesa
Angelus Vesta
Arleta
Arlington Heights
Arts District
Atwater Village
Baldwin Hills
Baldwin Hills/ Crenshaw
Baldwin Village
Baldwin Vista
Beachwood Canyon
Bel Air
Benedict Canyon
Beverly Crest
Beverly Glen
Beverly Grove
Beverly Hills Post Office
Beverly Park
Beverlywood
Boyle Heights
Brentwood
Brentwood Circle
Brentwood Glen
Broadway-Manchester
Brookside
Bunker Hill
Cahuenga Pass
Canoga Park
Canterbury Knolls
Carthay
Castle Heights
Central-Alameda
Central City
Century City
Chatsworth
Chesterfield Square
Cheviot Hills
Chinatown
Civic Center
Crenshaw
Crestwood Hills
Cypress Park
Del Rey
Downtown
Eagle Rock
East Gate Bel Air
East Hollywood
Echo Park
Edendale
El Sereno
Elysian Heights
Elysian Park
Elysian Valley
Encino
Exposition Park
Faircrest Heights
Fairfax
Fashion District
Filipinotown, Historic
Financial District
Florence
Flower District
Franklin Hills
Gallery Row
Garvanza
Glassell Park
Gramercy Park
Granada Hills
Green Meadows
Griffith Park
Hancock Park
H

In [62]:
print(losangeles_venues.shape)
losangeles_venues.head()

(4671, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Angelino Heights,34.07029,-118.2548,Halliwell Manor,34.069329,-118.254165,Performing Arts Venue
1,Angelino Heights,34.07029,-118.2548,Guisados,34.070262,-118.250437,Taco Place
2,Angelino Heights,34.07029,-118.2548,Eightfold Coffee,34.071245,-118.250698,Coffee Shop
3,Angelino Heights,34.07029,-118.2548,"Michael Jackson's ""Thriller"" House (and Tree)",34.069557,-118.254599,Historic Site
4,Angelino Heights,34.07029,-118.2548,Ototo,34.072659,-118.251740,Sake Bar


Let's check how many venues were returned for each neighborhood


In [63]:
losangeles_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Angelino Heights,26,26,26,26,26,26
Angelus Vesta,2,2,2,2,2,2
Arleta,3,3,3,3,3,3
Arlington Heights,11,11,11,11,11,11
Arts District,42,42,42,42,42,42
Atwater Village,37,37,37,37,37,37
Baldwin Hills,12,12,12,12,12,12
Baldwin Hills/ Crenshaw,39,39,39,39,39,39
Baldwin Village,9,9,9,9,9,9


#### Let's find out how many unique categories can be curated from all the returned venues


In [64]:
print('There are {} uniques categories.'.format(len(losangeles_venues['Venue Category'].unique())))

There are 366 uniques categories.


##  Analyze Each Neighborhood


In [65]:
# one hot encoding
losangeles_onehot = pd.get_dummies(losangeles_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
losangeles_onehot['Neighborhoods'] = losangeles_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [losangeles_onehot.columns[-1]] + list(losangeles_onehot.columns[:-1])
losangeles_onehot = losangeles_onehot[fixed_columns]

losangeles_onehot.head()

,Neighborhoods,ATM,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Library,College Quad,College Residence Hall,College Theater,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Medical School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Picnic Area,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Post Office,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car

In [66]:
losangeles_onehot.shape

(4671, 367)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [67]:
losangeles_grouped = losangeles_onehot.groupby('Neighborhoods').mean().reset_index()
losangeles_grouped.head(10)

,Neighborhoods,ATM,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Library,College Quad,College Residence Hall,College Theater,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Medical School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Picnic Area,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Post Office,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car

In [68]:
losangeles_grouped.shape

(196, 367)

 let's create a fucntion that returns the most common venues in each neighborhood

In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

let's create a dataframe and display the top 10 venues of each neighborhood


In [70]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = losangeles_grouped['Neighborhoods']

for ind in np.arange(losangeles_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(losangeles_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Angelino Heights,Mexican Restaurant,Taco Place,Record Shop,Pet Store,Motel,Breakfast Spot,Boxing Gym,Clothing Store,Cocktail Bar,Coffee Shop
1,Angelus Vesta,Music Venue,Fried Chicken Joint,Film Studio,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market
2,Arleta,Athletics & Sports,Taco Place,Arts & Entertainment,Yoshoku Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm
3,Arlington Heights,Theater,Restaurant,Mexican Restaurant,Donut Shop,Bakery,Korean Restaurant,Art Gallery,Entertainment Service,Grocery Store,Coffee Shop
4,Arts District,Cocktail Bar,Coffee Shop,Ice Cream Shop,Art Gallery,Italian Restaurant,Bookstore,Brewery,Seafood Restaurant,Grocery Store,Gun Range


## Cluster Neighborhoods


Run k means to cluster the neighborhoods

In [71]:
# set number of clusters to 5
kclusters = 5

losangeles_grouped_clustering = losangeles_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(losangeles_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [72]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

losangeles_merged = la_data

# merge losangeles_grouped with la_data to add latitude/longitude for each neighborhood
losangeles_merged = losangeles_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhood')
print(losangeles_merged.shape)
losangeles_merged.head()

(201, 14)


,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Angelino Heights,34.070290,-118.254800,2.0,Mexican Restaurant,Taco Place,Record Shop,Pet Store,Motel,Breakfast Spot,Boxing Gym,Clothing Store,Cocktail Bar,Coffee Shop
1,Angeles Mesa,18.285830,-66.800890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angelus Vesta,33.994983,-118.341117,2.0,Music Venue,Fried Chicken Joint,Film Studio,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market
3,Arleta,34.249050,-118.433490,2.0,Athletics & Sports,Taco Place,Arts & Entertainment,Yoshoku Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm
4,Arlington Heights,34.039890,-118.325160,2.0,Theater,Restaurant,Mexican Restaurant,Donut Shop,Bakery,Korean Restaurant,Art Gallery,Entertainment Service,Grocery Store,Coffee Shop


In [73]:
# We notice that 'Angeles Mesa' does not have a valid cluster label, so drop rows with nan value for 'Cluster Labels'
losangeles_merged.dropna(subset=["Cluster Labels"], axis=0, inplace =True)
print(losangeles_merged.shape)
losangeles_merged.head()


(196, 14)


,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Angelino Heights,34.070290,-118.254800,2.0,Mexican Restaurant,Taco Place,Record Shop,Pet Store,Motel,Breakfast Spot,Boxing Gym,Clothing Store,Cocktail Bar,Coffee Shop
2,Angelus Vesta,33.994983,-118.341117,2.0,Music Venue,Fried Chicken Joint,Film Studio,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market
3,Arleta,34.249050,-118.433490,2.0,Athletics & Sports,Taco Place,Arts & Entertainment,Yoshoku Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm
4,Arlington Heights,34.039890,-118.325160,2.0,Theater,Restaurant,Mexican Restaurant,Donut Shop,Bakery,Korean Restaurant,Art Gallery,Entertainment Service,Grocery Store,Coffee Shop
5,Arts District,34.041964,-118.236387,2.0,Cocktail Bar,Coffee Shop,Ice Cream Shop,Art Gallery,Italian Restaurant,Bookstore,Brewery,Seafood Restaurant,Grocery Store,Gun Range


In [74]:
# we change 'Cluster Labels' data type to int from float, so we can visulaize the clusters.
losangeles_merged['Cluster Labels'] = losangeles_merged['Cluster Labels'].astype(int)

Finally, let's visualize the resulting clusters


In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(losangeles_merged['Latitude'], losangeles_merged['Longitude'], losangeles_merged['Neighborhood'], losangeles_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


##  Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 0


In [76]:
cluster_zero = losangeles_merged.loc[losangeles_merged['Cluster Labels'] == 0, losangeles_merged.columns[[0] + list(range(4, losangeles_merged.shape[1]))]]
cluster_zero

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Beverly Crest,Other Great Outdoors,Yoshoku Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market
56,Faircrest Heights,Convenience Store,Yoshoku Restaurant,Film Studio,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market
134,Playa Vista,Food Truck,Gym,Office,Café,Park,Trail,Medical Center,Pizza Place,Golf Course,Plaza
152,South Robertson,Convenience Store,Salon / Barbershop,Yoshoku Restaurant,Filipino Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
160,Terminal Island,Food Truck,Yoshoku Restaurant,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market


In [77]:
print(cluster_zero['1st Most Common Venue'].value_counts())
print(cluster_zero['2nd Most Common Venue'].value_counts())
print(cluster_zero['3rd Most Common Venue'].value_counts())

Food Truck              2
Convenience Store       2
Other Great Outdoors    1
Name: 1st Most Common Venue, dtype: int64
Yoshoku Restaurant    3
Gym                   1
Salon / Barbershop    1
Name: 2nd Most Common Venue, dtype: int64
English Restaurant     1
Office                 1
Filipino Restaurant    1
Film Studio            1
Yoshoku Restaurant     1
Name: 3rd Most Common Venue, dtype: int64


#### Cluster 1

In [78]:
cluster_one = losangeles_merged.loc[losangeles_merged['Cluster Labels'] == 1, losangeles_merged.columns[[0] + list(range(4, losangeles_merged.shape[1]))]]
cluster_one

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,Jewelry District,Restaurant,Plaza,Diner,Yoshoku Restaurant,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
117,Nichols Canyon,Home Service,Plaza,Yoshoku Restaurant,Filipino Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
118,NoHo Arts District,Restaurant,Plaza,Diner,Yoshoku Restaurant,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
123,Old Bank District,Restaurant,Plaza,Diner,Yoshoku Restaurant,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop


In [79]:
print(cluster_one['1st Most Common Venue'].value_counts())
print(cluster_one['2nd Most Common Venue'].value_counts())
print(cluster_one['3rd Most Common Venue'].value_counts())

Restaurant      3
Home Service    1
Name: 1st Most Common Venue, dtype: int64
Plaza    4
Name: 2nd Most Common Venue, dtype: int64
Diner                 3
Yoshoku Restaurant    1
Name: 3rd Most Common Venue, dtype: int64


#### Cluster 2

In [82]:
cluster_two = losangeles_merged.loc[losangeles_merged['Cluster Labels'] == 2, losangeles_merged.columns[[0] + list(range(4, losangeles_merged.shape[1]))]]
cluster_two


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Angelino Heights,Mexican Restaurant,Taco Place,Record Shop,Pet Store,Motel,Breakfast Spot,Boxing Gym,Clothing Store,Cocktail Bar,Coffee Shop
2,Angelus Vesta,Music Venue,Fried Chicken Joint,Film Studio,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market
3,Arleta,Athletics & Sports,Taco Place,Arts & Entertainment,Yoshoku Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm
4,Arlington Heights,Theater,Restaurant,Mexican Restaurant,Donut Shop,Bakery,Korean Restaurant,Art Gallery,Entertainment Service,Grocery Store,Coffee Shop
5,Arts District,Cocktail Bar,Coffee Shop,Ice Cream Shop,Art Gallery,Italian Restaurant,Bookstore,Brewery,Seafood Restaurant,Grocery Store,Gun Range
6,Atwater Village,Pizza Place,Coffee Shop,Vietnamese Restaurant,Cuban Restaurant,Pet Store,Latin American Restaurant,Taco Place,Bookstore,Boutique,Liquor Store
7,Baldwin Hills,Pizza Place,Art Gallery,Sandwich Place,Mexican Restaurant,Fast Food Restaurant,Park,Video Store,Convenience Store,Liquor Store,Intersection
8,Baldwin Hills/ Crenshaw,Lingerie Store,Department Store,Sandwich Place,Southern / Soul Food Restaurant,Fast Food Restaurant,New American Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,Supplement Shop,Breakfast Spot
9,Baldwin Village,Mexican Restaurant,Coffee Shop,Restaurant,Park,Convenience Store,Office,IT Services,Flea Market,Farmers Market,Escape Room
10,Baldwin Vista,Mexican Restaurant,Coffee Shop,Restaurant,Park,Convenience Store,Office,IT Services,Flea Market,Farmers Market,Escape Room


In [83]:
print(cluster_two['1st Most Common Venue'].value_counts())
print(cluster_two['2nd Most Common Venue'].value_counts())
print(cluster_two['3rd Most Common Venue'].value_counts())

Coffee Shop                    21
Mexican Restaurant             18
Pizza Place                    13
Fast Food Restaurant            8
Trail                           6
Park                            5
Theater                         4
Sandwich Place                  4
Bar                             4
Korean Restaurant               4
Thai Restaurant                 4
Liquor Store                    3
Chinese Restaurant              3
Japanese Restaurant             3
Grocery Store                   3
Asian Restaurant                3
Café                            3
Italian Restaurant              3
Burger Joint                    2
Convenience Store               2
Music Venue                     2
Gym / Fitness Center            2
Food Truck                      2
Arts & Crafts Store             2
Gym                             2
Lingerie Store                  2
Cocktail Bar                    2
Fried Chicken Joint             2
Sushi Restaurant                2
Ice Cream Shop

#### Cluster 3

In [84]:
cluster_three = losangeles_merged.loc[losangeles_merged['Cluster Labels'] == 3, losangeles_merged.columns[[0] + list(range(4, losangeles_merged.shape[1]))]]
cluster_three

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,Griffith Park,Trail,Picnic Area,Mountain,Bus Stop,Scenic Lookout,Yoshoku Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
139,Rose Hills,Trail,Yoshoku Restaurant,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm
177,Victoria Park,Trail,Yoshoku Restaurant,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm


In [85]:
print(cluster_three['1st Most Common Venue'].value_counts())
print(cluster_three['2nd Most Common Venue'].value_counts())
print(cluster_three['3rd Most Common Venue'].value_counts())

Trail    3
Name: 1st Most Common Venue, dtype: int64
Yoshoku Restaurant    2
Picnic Area           1
Name: 2nd Most Common Venue, dtype: int64
Fast Food Restaurant    2
Mountain                1
Name: 3rd Most Common Venue, dtype: int64


#### Cluster 4

In [86]:
cluster_four = losangeles_merged.loc[losangeles_merged['Cluster Labels'] == 4, losangeles_merged.columns[[0] + list(range(4, losangeles_merged.shape[1]))]]
cluster_four

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Bel Air,Park,Road,Café,Yoshoku Restaurant,Fast Food Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
15,Beverly Glen,Park,Yoshoku Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market
30,Carthay,Entertainment Service,Park,Athletics & Sports,Fast Food Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm
32,Central-Alameda,Clothing Store,Park,Mobile Phone Shop,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm
46,East Gate Bel Air,Park,Road,Café,Yoshoku Restaurant,Fast Food Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
50,El Sereno,Pizza Place,Park,Trail,Empanada Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
52,Elysian Park,Park,Disc Golf,Playground,Yoshoku Restaurant,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
86,Jefferson Park,Neighborhood,Park,Market,Yoshoku Restaurant,Fast Food Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
112,Montecito Heights,Park,Market,Convenience Store,Yoshoku Restaurant,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Farm
137,Reseda,Park,Gym,Food,Fast Food Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop


In [87]:
print(cluster_four['1st Most Common Venue'].value_counts())
print(cluster_four['2nd Most Common Venue'].value_counts())
print(cluster_four['3rd Most Common Venue'].value_counts())

Park                     8
Clothing Store           1
Pizza Place              1
Entertainment Service    1
Neighborhood             1
Name: 1st Most Common Venue, dtype: int64
Park                  4
Yoshoku Restaurant    2
Road                  2
Market                1
Gym                   1
Disc Golf             1
Clothing Store        1
Name: 2nd Most Common Venue, dtype: int64
English Restaurant    2
Café                  2
Market                1
Playground            1
Convenience Store     1
Food                  1
Trail                 1
Mobile Phone Shop     1
Flower Shop           1
Athletics & Sports    1
Name: 3rd Most Common Venue, dtype: int64


## Results

We examine each cluster for the most common venues and also the existence and quantity of Ethiopian restaurant.                 
* Cluster 0 

This cluster has  5 neighborhood in it, and its 1st most common venues are 'Food Truck' with 2 counts, 'Convenience Store' with 2 counts, and 'Other Great Outdoors'. Since it does not have food venue other than food truck in its 1st most common venue, we consider it for further analysis to determine if it is the location to open an Ehiopian Restaurant. With regards to Ethiopian restaurant venue, this cluster has this venue as one of the top most common venue (1st thru 10th) in its neighborhoods except one, Playa Vista.                                                                                                                

* Cluster 1

This cluster has 4 neighborhoods in it, and its 1st most common venues are 'Restaurant' with 3 counts and one 'Home Service'.    Since restaurant is the 1st most common venue in this cluster and it has 'Ethiopian Restaurant' as its 7th most common venue in  all the neighborhoods, we will not consider this cluster for further analysis and make the determination that this is not the    best loaction to open an Ethiopian restaurant in Los Angeles.


* Cluster 2

This cluster has more than 100 neighborhoods in it and it is the largest cluster by far. The 1st most common venue in this cluster are 21 'Coffee Shops', 18 'Mexican Restaurant', 13 'Pizza Place', 8 'Fast Food Restaurant', and so on. This cluster has many other food venues in its top 10 most common venues. As far as the specific venue 'Ethiopian Restaurant', this cluster has multiple neighborhoods that has this venue in the top most common venue (1st thru 10th), for instance, Little Ethiopia has it as 1st most common venue, Carthay has it as its 6th most common venue. Therefore, we will not consider this cluster as suitable location to open an Ethiopian restaurant.

* Cluster 3

This cluster has 3 neighborhoods in it, and its 1st most common venue is 'Trail'. This cluster also has 'Ethiopian Restaurant' venue as one of the top 10 most common venue in all of its neighborhoods. The three neighborhoods all have 10 or less venues in them, therefore, because of the existence of the venue 'Ethiopian Restaurant' as top common venue in all of the neighborhoods and the small number venues that exist in total indicating not a central location; we will not consider this cluster.

* Cluster 4

This cluster has 12 neighborhoods in it. The 1st most common venues are 'Park' with 8 counts, 'Clothing Store', 'Pizza Place', 'Entertainment Service' with one count each. Even though, no food venue is in the 1st most common venue, 2nd and 3rd most common venues have food venues including restaurants. With regards to the venue 'Ethiopian Restaurant', all 12 neighborhoods in this cluster have this venues in one the top 10 most common venue. Therefore, we will not consider to open an Ethiopian restaurant.

Based on the analysis of each cluster, cluster 0 is the best location to open an Ethiopian restaurant. Since all the neighborhoods have 'Ethiopian Restaurant' venue in the top most common venue except 'Playa Vista', we will pick 'Playa Vista' neighborhood of Los Angeles to be the best location to open an Ethiopaina restaurant.